# Rewrite Zoning Lot Yard Cutter Geometry Logic

2021-09-21

This notebook outlines efforts to improve height/setback cutters used for zoning envelope creation.

In [1]:
import os
import sys
import pickle

In [2]:
# add udtools library to path
home_dir = os.environ.get('HOME')
udtools_module_path = os.path.join(home_dir, 'git', 'ud-tools-core', 'udtools')

if udtools_module_path not in sys.path:
  sys.path.append(udtools_module_path)

In [3]:
from archtools.geometry.common import get_edges, get_vertices
from archtools.geometry.edge import get_endpoints
from archtools.geometry.reference import world_xy
from OCC.Core.GeomAPI import geomapi_To2d
from OCC.Core.BRep import BRep_Tool
from archtools.geometry.conversion import wire_to_curve
from archtools.geometry.wire import is_ccw, get_vertices as get_wire_vertices
from archtools.connectors.wkt import coords_to_curve, coords_to_edge
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve, BRepAdaptor_CompCurve
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire

In [4]:
# load test geoms, saved from zoninglot notebook 2021-09-21
# with open('./assets/20210921_edge850.pickle', 'rb') as f:
#   edge = pickle.load(f)
  
# with open('./assets/20210921_edge849.pickle', 'rb') as f:
#   edge = pickle.load(f)

# with open('./assets/20210921_edge3192.pickle', 'rb') as f:
#   edge = pickle.load(f)

with open('./assets/20210921_edge846.pickle', 'rb') as f:
  edge = pickle.load(f)

# with open('./assets/20210921_edge105.pickle', 'rb') as f:
#   edge = pickle.load(f)
  
# with open('./assets/20210921_edge106.pickle', 'rb') as f:
#   edge = pickle.load(f)

with open('./assets/20210921_bounds202285.pickle', 'rb') as f:
  bounds = pickle.load(f)

In [5]:
edge_pnts = get_vertices(edge, topo=False)

In [6]:
edge_endpoints = [p.Coord() for p in edge_pnts]
edge_endpoints

[(-1722.04851775966, 513.887860742863, 0.0),
 (-1714.42912631831, 509.663799730595, 0.0)]

In [7]:
# get edge as curve
# method 1: BRep_Tool
# edge_astoolcurve = BRep_Tool.Curve(edge)[0]
# edge_as2dtoolcurve = geomapi_To2d(edge_astoolcurve, world_xy) 
# works but is in a different coordinate system?

# method 2: BRepAdaptor_Curve
# edge_ascurve = BRepAdaptor_Curve(edge)
#edge_as2dcurve = geomapi_To2d(edge_ascurve, world_xy) # doesn't work

# method 3:
edge_aswire = BRepBuilderAPI_MakeWire(edge).Shape()
coords = [p.Coord()  for p in get_wire_vertices(edge_aswire, topo=False)]
edge_ascurve = coords_to_curve(coords)

Seems like `geomapi_To2d` isn't compatible with the result of `BRepAdaptor_Curve`. The `BRep_Tool` approach above seems to not return a valid curve for some reason (start/end parameters are positve/negative infinity). Is it possible to rewrite `curve_fit_to_bounds` to use 3d curves instead of 2d?

In [8]:
ec = edge_ascurve
curve_endparams = [ec.FirstParameter(), ec.LastParameter()]
curve_endpoints = [ec.Value(p) for p in curve_endparams]
curve_endpoint_coords = [p.Coord() for p in curve_endpoints]
curve_endpoint_coords # should return same as edge_endpoints

[(-1722.04851775966, 513.887860742863, 0.0),
 (-1714.42912631831, 509.663799730595, 0.0)]

In [9]:
# first and last params, 0, 21.4
# ec.Value(30).Coord()
# trimmed = ec.Trim(-10, 30, 0.1)

Looks good. Now look at the `bounds` Wire, see if we can verify that the curve equivalent can be reliably generated.

In [10]:
# first, preview wire vertices as point coordinates
bounds_pts = get_vertices(bounds, topo=False)
bounds_coords = [p.Coord() for p in bounds_pts]
bounds_coords

[(-1722.04851775966, 513.887860742863, 0.0),
 (-1672.43083339534, 602.703972784337, 0.0),
 (-1672.43083339534, 602.703972784337, 0.0),
 (-1654.90727191768, 592.968755689915, 0.0),
 (-1654.90727191768, 592.968755689915, 0.0),
 (-1636.18092447123, 582.565561262425, 0.0),
 (-1636.18092447123, 582.565561262425, 0.0),
 (-1685.78056484065, 493.781606165227, 0.0),
 (-1704.51608663402, 504.168402162846, 0.0),
 (-1685.78056484065, 493.781606165227, 0.0),
 (-1714.42912631831, 509.663799730595, 0.0),
 (-1704.51608663402, 504.168402162846, 0.0),
 (-1722.04851775966, 513.887860742863, 0.0),
 (-1714.42912631831, 509.663799730595, 0.0)]

For reference:

```wkt
'POLYGON((-1636.18092447123 582.565561262425,-1654.90727191768 592.968755689915,-1672.43083339534 602.703972784337,-1722.04851775966 513.887860742863,-1714.42912631831 509.663799730595,-1704.51608663402 504.168402162846,-1685.78056484065 493.781606165227,-1636.18092447123 582.565561262425))'
```

Note this is the wkt boundary returned directly from PostGIS using `ST_ForcePolygonCCW` to enforce a counter-clockwise direction, which is what OCC uses as "FORWARD"

This seems wrong, will it work to use `ShapeFix_Wire`

The orientation of our wire should be FORWARD by default, this is coded as 0 in python and means the wire direction is counter-clockwise. We can check this using the `is_ccw` function from `archtools.geometry.wire` internally, this checks if the wire is oriented correctly by making sure it has positive area.

In [11]:
bounds.Orientation() # should be 0

0

In [12]:
# check orientation of the wire by area
is_ccw(bounds)

True

In order to make use of available intersection methods available for OCC geometries, we need geometry equivalents of our Edge and Wire (both are topologies right now). The Edge → Curve case is more straightforward (see above) but Wire → Curve isn't so easy. A few possible methods:

In [13]:
[p.Coord() for p in get_wire_vertices(bounds, topo=False)]

[(-1722.04851775966, 513.887860742863, 0.0),
 (-1672.43083339534, 602.703972784337, 0.0),
 (-1654.90727191768, 592.968755689915, 0.0),
 (-1636.18092447123, 582.565561262425, 0.0),
 (-1685.78056484065, 493.781606165227, 0.0),
 (-1704.51608663402, 504.168402162846, 0.0),
 (-1714.42912631831, 509.663799730595, 0.0),
 (-1722.04851775966, 513.887860742863, 0.0)]

In [14]:
# method 1: BRep_Tool
# bounds_astoolcurve = BRep_Tool.Curve(bounds)[0] # won't work, BRep_Tool is for edges

# method 2: BRepAdaptor_CompCurve
# bounds_ascurve = BRepAdaptor_CompCurve(bounds) # compcurve, not curve
# lets you get params etc but won't work in other algorithms

# method 3: rebuild as new polygonal curve from vertices
coords = [p.Coord()  for p in get_wire_vertices(bounds, topo=False)]
bounds_ascurve = coords_to_curve(coords)

In [15]:
# generalize this in a function
def curve_from_wire_vertices(wire):
  coords = [p.Coord()  for p in get_vertices(wire, topo=False)]
  curve = coords_to_curve(coords)
  return curve

In [16]:
# and preview values along the generated curve
bc = bounds_ascurve
bounds_endparams = [bc.FirstParameter(), bc.LastParameter()]
bounds_endpoints = [bc.Value(p) for p in bounds_endparams]
bounds_endpoint_coords = [p.Coord() for p in bounds_endpoints]
bounds_endpoint_coords # should return same as edge_endpoints above

[(-1722.04851775966, 513.887860742863, 0.0),
 (-1722.04851775966, 513.887860742863, 0.0)]

In [17]:
# start and endpoint are the same since it's a closed curve, but it looks OK
# what about knots?
bc.NbKnots()

8

In [18]:
# should be parameterized from 0 to 1
bc.FirstParameter()
bc.LastParameter()

1.0

In [19]:
# check a point in the middle
bc.Value(0.5).Coord()

(-1636.196965469247, 582.5744726436573, 0.0)

Now comes the tricky part, we want to make sure our functions to trim/extend an offset edge lying partially inside the bounding polygon to meet either edge are still working.

In [20]:
import math
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir, gp_Ax1, gp_Pnt2d
from OCC.Core.gce import gce_MakeLin
from OCC.Core.GeomAPI import (
  GeomAPI_ExtremaCurveCurve, 
  GeomAPI_ProjectPointOnCurve,
  geomapi_To3d,
  geomapi_To2d,
)
from OCC.Core.Geom2dAPI import Geom2dAPI_ExtremaCurveCurve, Geom2dAPI_InterCurveCurve
from archtools.geometry.conversion import pts_to_curve, curve_to_wire
from archtools.geometry.common import moved, get_vertices
from archtools.geometry.reference import pos_z, world_xy
from archtools.geometry.measure import area
from archtools.geometry.curve import match_curve_direction
from OCC.Core.Geom import Geom_Curve, Geom_OffsetCurve, Geom_Line
from OCC.Core.Geom2d import Geom2d_OffsetCurve, Geom2d_Curve
from OCC.Core.IntCurvesFace import IntCurvesFace_ShapeIntersector
from OCC.Core.BRepExtrema import BRepExtrema_DistShapeShape
from OCC.Core.BRepLib import BRepLib_MakeFace
from OCC.Core.IntCurvesFace import IntCurvesFace_Intersector


First offset the edge:

In [21]:
# this is potentially useful but easier to use curves
# planar offset of an edge
# def planar_offset(e, distance):
#   ec = BRepAdaptor_Curve(e)
#   p = [ec.FirstParameter(), ec.LastParameter()]
#   p_mid = (p[0] + p[1] / 2)

#   pnt = gp_Pnt()
#   vec = gp_Vec()
#   ec.D1(p_mid, pnt, vec)

#   vec.Rotate(gp_Ax1(pnt, pos_z), -math.pi/2)
#   vec.Multiply(distance)

#   return moved(e, vec.Coord())

In [22]:
# wire_setback = planar_offset(edge, -30)
# wire_setback_curve = curve_from_wire_vertices(wire_setback)

In [23]:
# using curves...
  # match_curve_direction(lotline_2d, bounds_2d, z=False)

match_curve_direction(edge_ascurve, bounds_ascurve, z=True)

pnt = gp_Pnt()
vec = gp_Vec()
edge_ascurve.D1(0.5, pnt, vec)
# vec.Rotate(gp_Ax1(pnt, pos_z), -math.pi/2)
# offset_dir = gp_Dir(vec)
offset_dir = pos_z
setback_curve = Geom_OffsetCurve(edge_ascurve, 5, offset_dir)
# setback_curve = geomapi_To3d(setback_2dcurve, world_xy)

wire_setback_curve = setback_curve
wire_setback = curve_to_wire(wire_setback_curve)

c1 was reversed!


In [24]:
# def extended_point_at_param(c1, c2, param):
#   """Used internally by curve_fit_to_bounds.

#   Tries to find and return point on c2 nearest to the provided param on c1.
#   If unsuccessful, returns point at the param on c1.
#   """

#   result = c1.Value(param)
#   print(result.Coord())

#   test_from = -1 if param == 0 else 0.5
#   test_to = 0.5 if param == 0 else 2
#   ex = GeomAPI_ExtremaCurveCurve(c1, c2, test_from, test_to, 0, 1)
#   ex_count = ex.NbExtrema()

#   if ex_count == 0: return result

#   c1_pt = gp_Pnt() # these Pnt2d in 2d version
#   c2_pt = gp_Pnt()
#   ex.NearestPoints(c1_pt, c2_pt)
#   print(c2_pt.Coord())
#   print(ex_count)

#   return c2_pt

In [25]:
# alternate version of above using ProjectPointOnCurve
# CURRENTLY USED
def extended_point_at_param(c1, c2, param):
  """Tries to find nearest point on c2 to provided param on c1"""
  p = c1.Value(param)
  projector = GeomAPI_ProjectPointOnCurve(p, c2)
  result = projector.NearestPoint()
  return result

In [26]:
# another approach using Geom2dAPI_InterCurveCurve
# to get intersections of edge as infinite line and curve

def trim_curve_to_bounds_2(curve, bounds):
  c = curve
  ln = gce_MakeLin(c.Value(0), c.Value(1)).Value()
  ln_curve = Geom_Line(ln)
  
  line_2d = geomapi_To2d(ln_curve, world_xy)
  bounds_2d = geomapi_To2d(bounds_ascurve, world_xy)
  
  intersector = Geom2dAPI_InterCurveCurve(line_2d, bounds_2d, 0.001)
  
  npts = intersector.NbPoints()
  pts = []

  for i in range(1, npts + 1):
    pt = intersector.Point(i)
    pts.append(pt)
    
  # to do, sort from center of curve line?

  coords = [p.Coord() for p in pts]
  result = coords_to_curve([coords[0], coords[-1]], z=False)
  result_3d = geomapi_To3d(result, world_xy)
  
#   result_asedge = BRepBuilderAPI_MakeEdge(result_3d).Shape()

  return result_3d


In [27]:
# ln = to_line_from_vertices(wire_setback)
wsc = wire_setback_curve
ln = gce_MakeLin(wsc.Value(0), wsc.Value(1)).Value()
ln_curve = Geom_Line(ln)
# ip = intersection_points(ln_curve, bounds_ascurve)

In [28]:
line_2d = geomapi_To2d(ln_curve, world_xy)
bounds_2d = geomapi_To2d(bounds_ascurve, world_xy)

In [29]:
# line_2d.Value(-100000).Coord() # when the line gets converted it stops being infinite?
# or at least gets reparameterized from 0 to 1, others are still available but may
# need to be set manually on the intersector?
# line_2d.

In [30]:
intersector = Geom2dAPI_InterCurveCurve(line_2d, bounds_2d, 0.001)

In [31]:
npts = intersector.NbPoints()
pts = []

for i in range(1, npts + 1):
  pt = intersector.Point(i)
  pts.append(pt)
  print(pt.Coord())
#   if len(pts) == 0:
#     pts.append(pt)
#   elif min([pt.Distance(p) for p in pts]) < 1.0:
#     pts.append(pt)

(-1683.3420389918494, 498.1465968758601)
(-1719.6099541197455, 518.2529121199617)


In [32]:
coords = [p.Coord() for p in pts]
coords
result_edge = coords_to_edge(coords, z=False)

In [33]:
result_edge

<class 'TopoDS_Edge'>

In [34]:
result = trim_curve_to_bounds_2(wire_setback_curve, bounds_ascurve)
# works when returning edge

In [35]:
result.Value(1).Coord()

(-1719.6099541197455, 518.2529121199617, 0.0)

In [36]:
# copy of curve_fit_to_bounds from archtools.geometry
def trim_curve_to_bounds(curve, bounds):
  """Where curve and bounds are coplanar 3D OCC Curves.
  Bounds is expected to be closed and at least part of curve
  needs to be within bounds. Attempts to trim/extend endpoints
  of curve to meet bounds.
  """

  start = curve.Value(0)
  end = curve.Value(1)
  start_ext = extended_point_at_param(curve, bounds, 0)
  end_ext = extended_point_at_param(curve, bounds, 1)

  print([p.Coord() for p in [start, start_ext, end, end_ext]])
  # check that the start and end extension points aren't the same
  ext_are_diff = start_ext.Distance(end_ext) > 1.0 # threshold of 1 ft apart?

  if start.Distance(start_ext) > 0.0:
    start = start_ext

  if end.Distance(end_ext) > 0.0 and ext_are_diff:
    end = end_ext

  result = pts_to_curve([start, end], z=True)

  return result

In [37]:
# extended_point_at_param(edge_ascurve, bounds_ascurve, 0.0).Coord()
trimmed = trim_curve_to_bounds(wire_setback_curve, bounds_ascurve)

[(-1712.0048308849089, 514.0367611586881, 0.0), (-1714.42912631831, 509.663799730595, 0.0), (-1719.6242223262589, 518.2608221709562, 0.0), (-1719.6099800804784, 518.2528656500104, 0.0)]


In [38]:
# def fit_to_bounds(edge, bounds):
#   """Extend/trim ends of edge to boundary wire."""

#   # if type(curve) == Geom2d_OffsetCurve:
#   #   curve_2d = curve
#   # else:
#   #   curve_2d = geomapi_To2d(curve, world_xy)

#   # start = curve_2d.Value(0)
#   # end = curve_2d.Value(1)
#   (e_0, e_1) = get_vertices(edge, topo=True)

#   extrema = BRepExtrema_DistShapeShape()
#   extrema.SetDeflection(20)
#   extrema.LoadS1(edge)
#   extrema.LoadS2(bounds)
#   extrema.Perform()
    
#   return extrema.PointOnShape2(1)
  
#   start_ext = extrema_nearest_pt(curve, bounds, 0)
#   end_ext = extrema_nearest_pt(curve, bounds, 1)

#   for p in [start, start_ext, end, end_ext]:
#     print(p.Coord())

#   ext_are_diff = start_ext.Distance(end_ext) > 0.0

#   if start.Distance(start_ext) > 0.0:
#     start = start_ext

#   if end.Distance(end_ext) > 0.0 and ext_are_diff:
#     end = end_ext

#   result_2d = pts_to_curve([start, end], z=False)
  # result_3d = geomapi_To3d(result_2d, world_xy)

#   return result_2d


In [39]:
# edge as curve
#ec

# bounds as curve
# edge

In [40]:
# check for intersections based on line and bounds as face
# from archtools.geometry.edge import to_line_from_vertices
# from archtools.geometry.common import get_vertices
# from OCC.Core.gp import gp_Lin

# def check_intersections(line, bounds):
#   """intersection of line and Wire bounds"""
  
#   face = BRepLib_MakeFace(bounds).Face()
#   intersector = IntCurvesFace_Intersector(face, 0.1, True, False)
#   intersector.Perform(ln, 0, float("+inf"))
#   face = intersector.Face()
#   num_results = intersector.NbPnt()
#   print(intersector.Transition(1))
# #   intersector.SortResult()
#   # where State is TopAbs_ON (on boundary of face)
  
#   return (face, ec, num_results, intersector.IsParallel())

In [41]:
# ln = to_line_from_vertices(edge_aswire)
# check_intersections(ln, bounds)

`IntCurvesFace_Intersector` doesn't work if the line is parallel to the face.

In [42]:
# # test on newly created arbitrary geoms
# from OCC.Core.gce import gce_MakeLin
# from archtools.connectors.wkt import coords_to_polygon
# from archtools.geometry.common import get_vertices

# poly_list = [
#   (0, 0, 0),
#   (2, 0, 0),
#   (2, -1, 0),
#   (0, -1, 0),
#   (0, 0, 0)
# ]
# bounds = coords_to_polygon(poly_list)
# ln = gce_MakeLin(gp_Pnt(1, 1, 0), gp_Pnt(1, -2, 0)).Value()

# verts = get_vertices(bounds, topo=False)
# coords = [v.Coord() for v in verts]
# coords

In [43]:
# ln_wire = BRepBuilderAPI_MakeEdge(ln, gp_Pnt(1, 1, 0), gp_Pnt(1, -2, 0)).Edge()

In [44]:
# (f, l, num) = fit_to_bounds(ln, bounds)
# num

not sure this approach will work

## Visual Debugger

In order to check/validate geometry, it's helpful to be able to see it.

In [45]:
import ipywidgets as widgets
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [54]:
# initialize renderer
renderer = JupyterRenderer()

In [55]:
renderer.DisplayShape(edge, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)

In [48]:
# def trimmed_to_edge(t):
#   p_0 = t.FirstParameter()
#   p_1 = t.LastParameter()
#   pts = [t.Value(p) for p in [p_0, p_1]]
#   print([p.Coord() for p in pts])
#   e = BRepBuilderAPI_MakeEdge(*pts)
#   return e.Shape()  

In [56]:
renderer.DisplayShape(bounds)

In [57]:
# check the setback edge is looking ok prior to trim/extend step
renderer.DisplayShape(wire_setback)

In [51]:
# check that the curve created from our original wire looks ok
# bounds_curve = BRepBuilderAPI_MakeEdge(bounds_ascurve).Shape()
# renderer.DisplayShape(bounds_curve)

In [52]:
# check that the curve created from our original wire looks ok
# trimmed_asedge = BRepBuilderAPI_MakeEdge(trimmed).Shape()
# renderer.DisplayShape(trimmed_asedge)

In [58]:
# check edge from alternate trim/extend algorithm
# based on 2d curve/curve intersection
result_asedge = BRepBuilderAPI_MakeEdge(result).Shape()
renderer.DisplayShape(result_asedge)